# Introduction

In [2]:
# The idea of this competition, is to predict sales for mnay product families for stores in ecuador.
#Train and test share: date, store_nbr, family(product_family), onpromotion (number of products of the family that were on promotion)
#Train also has the sales.
#Test is what we are trying to predict in the competition

#Stores contains store metadata, we might want to join that data into our train and test tables to improve predictions if they are not redundant with the store_nbr (aka if at least some of the stores share their values).
 #stores metadata
#type and cluster are not redundant. see stores_df[['type', 'cluster']].drop_duplicates()#.pivot(columns='cluster')

#Daily oil price, keep in mind oil price affects ecuador economical health.
#We probably want to join it into our dataframe, but keep in mind it seems to have some null values. We probably want to fill downwards so that it takes its value from the previous date but well have to see. We could also average between the next and previous day.


#While its not clear it seems to be the total transactions of the store on that day


#A table containing holiday events. 
#its worth noting that some events were transfered in date so we need to take into account hte actual date. transfered column True means that the holiday wasnt actually celebrated that date.
#Also, keep in mind that some seem to be regional and some local. This could mean that we need to take into account the region in considering them.
#Some special_days were of type bridge, meaning  they are extra days added to the same holiday.
#This are often compensated by making some not working days(ie saturday) working days. This are of type workday.
#holiday_type additional means that it isnt an actual holiday, but actually the extension of one.

#Additonally to these tables wages on the public sector are paid on the 15th and last day of each month which could affect supermarket sales
#A great earthquiake struck ecuador on april 16 2016 which resulted in donations possibly affecting sales.


#TLDR we need to add many features to the train and test df.
#First lets make some column names easier to understand. Specially after joining.

In [3]:
import pandas as pd
import data_preparation as data_prep
#import tensorflow as tf
#import keras
#from tqdm import tqdm

In [4]:
#Reload all modules imported with %aimport every time before executing the Python code typed. https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload explicit
%aimport data_preparation 

pd.set_option('display.max_rows', 1000)
#TODO move the eda process to another file? Might need to move the processing functions to an auxiliary file aswell.

# Prepare the dataset for the pipeline

Please note that I intentionally merge the parts of the dataset outside the pipeline, so the pipeline will remain usable even if the data is provided in a different format, for example in a unified table. or as a manually created df.
TODO consider this again since its not a valid reason, considering that pipeline slicing is possible.

In [30]:
train_df, elements_to_predict_df, stores_df, oil_df, transactions_df, special_days_df, sample_submission_df = data_prep.rename_raw_dfs_cols(*data_prep.get_raw_dfs())
train_y_truth:pd.Series = train_df.pop('sales')

In [35]:
pipeline = data_prep.create_pipeline(stores_df, oil_df, transactions_df, special_days_df, verbose=False)
processed_train_df = pipeline.fit_transform(train_df)
processed_elements_to_predict = pipeline.transform(elements_to_predict_df)

display(pipeline)
display(processed_train_df.columns)
display(processed_train_df.head(5))
#TODO use the outpout tuple of rename dfs as input without keyword args for the merger of the pipeline. Modify the merge function acrodingly.

AttributeError: 'tuple' object has no attribute 'merge'

In [ ]:
from sklearn.model_selection import cross_val_score
import optuna

def objective(trial):
    # Define the hyperparameters
    hyperparams = { #When using pipelines, you need to prefix the parameters depending on which part of the pipeline they refer to with the name of the respective component
        'regressor__fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'regressor__normalize': trial.suggest_categorical('normalize', [True, False]),
    }

    # Set the hyperparameters to the pipeline
    fold_pipeline = data_prep.create_pipeline().set_params(**hyperparams)

    # Perform cross-validation and return the mean score
    scores = cross_val_score(fold_pipeline, features_df, target_df, cv=5, scoring='neg_mean_squared_error')
    
    #Store additional results for the trial
    trial.set_user_attr('cv_scores', scores)

    return scores.mean()

sampler = optuna.samplers.TPESampler() #The TPESampler is a class in Optuna that implements the Tree-structured Parzen Estimator (TPE) algorithm, which is a kind of genetic algorithm.
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)

for i, trial in enumerate(study.trials):
    print(f'Trial {i}:')
    print(f'  Params: {trial.params}')
    print(f'  Scores: {trial.user_attrs["cv_scores"]}')

best_params = study.best_params
print(best_params)

pipeline = data_prep.create_pipeline().set_params(best_params)
#it could be a good idea here to do cv (maybe with cv_predict) and analyze the predictions during cv in order to find where it fails, and compare it to other models to make enssambles.
#Maybe what we could compare is the difference in the prediction for each value, and sum the difference in those diferences between the models.
#The number of models in the ensamble could be tuned in another trial.
#The weights in voting in the ensamble could be tuned in another trial.

pipeline.set_params(**best_params)
pipeline.fit(train_df, dataset_y)

predictions = pipeline.predict(elements_to_predict_df)


# Analyze the performance
# This part depends on what kind of analysis you want to do.
# For example, you can calculate the prediction error for each sample:
errors = target_df - pipeline.predict(features_df)